In [1]:
import folium, json
import pandas as pd
import numpy as np
from folium import plugins
from folium.plugins import Search
import folium.features 
from plotly.subplots import make_subplots
# import plotly.express as px
import plotly.graph_objects as go
# import vincent
# from ipywidgets import AppLayout, Button, Layout

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import requests
def death_nepal():
    url = "https://nepalcorona.info/api/v1/data/nepal"
    response = requests.get(url)
    todos = json.loads(response.text)
    return todos

# # Corona Virus Dashboard for Nepal

In [4]:
data=death_nepal()

tested_positive = str(data['tested_positive'])
tested_total=str(data['tested_total'])
in_isolation=str(data['in_isolation'])
pending_result=str(data['pending_result'])
recovered=str(data['recovered'])
deaths=str(data['deaths'])
updated_at=data['latest_sit_report']['updated_at']




# display(HTML('<div style="float: left;width: 20%;padding: 0 10px;><div style="float: left;width: 25%;padding: 0 10px;"><divstyle="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2); padding: 16px; text-align: center; background-color: #f1f1f1;"><h3>'+tested_positive+'</h3><p>Tested Positive</p><p>Some text</p></div></div><div style="float: left;width: 25%;padding: 0 10px;"><div><h3>Card 1</h3><p>Some text</p><p>Some text</p></div></div></div>'))
display(HTML('<div style="width: 100%;display: block;margin-bottom: 20px;"><div style="float: left;width: 20%;padding: 0 10px;"><div style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);padding: 16px;text-align: center;background-color: #ff5722;"><h1>'+tested_positive+'</h1><p>Total Positive</p></div></div><div style="float: left;width: 20%;padding: 0 10px;"><div style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);padding: 16px;text-align: center;background-color: #39065a;" ><h1>'+tested_total+'</h1><p>Total Tested</p></div></div><div style="float: left;width: 20%;padding: 0 10px;"><div style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);padding: 16px;text-align: center;background-color: #6a0572;"><h1>'+recovered+'</h1><p>Recovered</p></div></div><div style="float: left;width: 20%;padding: 0 10px;"><div style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);padding: 16px;text-align: center;background-color: #9a0f98;"><h1>'+deaths+'</h1><p>Deaths</p></div></div><div style="float: left;width: 20%;padding: 0 10px;"><div style="box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);padding: 16px;text-align: center;background-color: #ea0599;"><h1>'+in_isolation+'</h1><p>Isolation</p></div></div></div>')) 
display(HTML('<p>Data Source: <a href="https://www.mohp.gov.np/en/">MOHP </a> Updated :'+updated_at+'</p>'))             


In [9]:
with open('nepal.geojson') as f:
    nepal_districts_geo=json.load(f)
    
for i in nepal_districts_geo['features']:
    i['id']=i['properties']['DISTRICT']

cases_plot_data=pd.read_csv('test_data_cases.csv')
dis_mun = pd.read_csv('nepal_dis_mun_codes.csv')
#code to municipality
def get_district(codes):
    dr = dis_mun.loc[dis_mun['VDC_code'] == codes] 
    if dr.empty == True:
        mun = "Not Data"
    else:
        drl = dr.values.tolist()
        mun = drl[0][5]
    return (mun)

#nepal-dash-search
md = folium.Map(location=[28.277433,83.581612], tiles="CartoDB positron",zoom_start=7, max_zoom=32, min_zoom=7, width='100%', height='100%')
plugins.ScrollZoomToggler().add_to(md)
#Adding minimap
minimap = plugins.MiniMap()
md.add_child(minimap)

mcg = folium.plugins.MarkerCluster(control=False)
md.add_child(mcg)

g2 = folium.plugins.FeatureGroupSubGroup(mcg, 'Perosn infectious Mapping')
url = "https://data.nepalcorona.info/api/v1/covid"
response = requests.get(url)
infected_data = json.loads(response.text)
victim_mun=[]
for i in range(len(infected_data)):
    victim_mun.append(infected_data[i]['municipality'])

url_mun ="https://data.nepalcorona.info/api/v1/municipals/"
response_mun = requests.get(url_mun)
total_mun = json.loads(response_mun.text)

for i in range(len(victim_mun)):
    for j in range(len(total_mun)):
        if(total_mun[j]['id']==victim_mun[i]):
            municipality_name = total_mun[j]['title_en']
            typee = total_mun[j]['type']
            folium.Marker([infected_data[i]['point']['coordinates'][1], infected_data[i]['point']['coordinates'][0]],
                      tooltip=''+infected_data[i]['gender'].upper()+' Age '+str(infected_data[i]['age'])+' .',
                      popup = ''+str(typee)+': <span style="color:red"><b> '+str(municipality_name)+'</b></span> !',
                      icon=folium.Icon(color='red', icon='info-sign')
                 ).add_to(g2)   


md.add_child(g2)
stategeo=folium.GeoJson(
    nepal_districts_geo,
    name='Map of Nepal'
).add_to(md)
statesearch = plugins.Search(position='topright',
    layer=stategeo,
    geom_type='Polygon',
    placeholder='Search for a District',
    collapsed=False,
    search_label='DISTRICT',
    weight=6
).add_to(md)

folium.LayerControl().add_to(md)
md
# md.save('nepal-live.html')

# District & Municipality Victims of Nepal 

In [8]:
#widget-table-final
#table of data
df2 = pd.read_csv('https://raw.githubusercontent.com/mepesh/covid19-dashboard-voila/master/notebooks/test_data_cases.csv')
df1 = df2[['district','cases','isolation','death','recovered']]
#graph
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
#column name lowercase
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)

confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})


x_data=np.array(list(confirmed_df.iloc[:,5:].columns))
y_data_confirmed=np.sum(np.asarray(confirmed_df[confirmed_df['country']=="Nepal"].iloc[:,5:]), axis=0)
y_data_death=np.sum(np.asarray(death_df[death_df['country']=="Nepal"].iloc[:,5:]), axis=0)

#creating subplots
fig = make_subplots(
    rows=2, cols=2,column_widths=[0.4, 0.6],horizontal_spacing=0.05,vertical_spacing=0.10,
    specs=[[{"type":"table","rowspan":2},{"rowspan": 2}],
          [{"type":"table"},None]], 
    subplot_titles=("Nepal Victims by District","Scatter Plot of NEPAL by Date [Cases & Deaths]")
)
fig.add_trace(
    go.Table(
        columnwidth = [35,20,20,20,25],
        header=dict(
            values=["District", "Cases", "Isolation",
                    "Deaths", "Recovered"],
            line_color='darkslategray',
            fill_color='grey',
            font=dict(color='white', size=12),
            align="left"
        ),
        cells=dict(
            values=[df1[k].tolist() for k in df1.columns[0:]],fill_color='lavender',
            font=dict(size=12),
            height=25,
            align = "left")
    ),
    row=1, col=1
)
fig.append_trace(go.Scatter(
    y=y_data_confirmed, x=x_data,
    mode='lines+markers',
    line_color='rgb(128, 0, 128)',
    name='TotalCases',
), 1, 2)
fig.append_trace(go.Scatter(
    y=y_data_death, x=x_data,
    mode='lines+markers',
    line_color='rgb(0.98,0.00,0.00)',
    name='Deaths',
), 1, 2)

fig.update_layout(
    height=400,
    showlegend=False,    
)

fig.show()
